In [1]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
only_ans=False
from torch.utils.data import Dataset
import torch
from tqdm import tqdm
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
class QADataset(Dataset):
    def __init__(self, file_path, tokenizer, block_size=2048*2,only_ans=True):
        print("QADataset")
        self.examples = []
        self.masked_examples = []
        stand_for_tokens = tokenizer.encode("<|start_header_id|>assistant<|end_header_id|>", add_special_tokens=False)
        print(stand_for_tokens)
        #print(stand_for_tokens)
        self.text=[]
        with open(file_path, encoding="utf-8") as f:
            text = f.read()
        
        qa_pairs = text.split("<|end_of_text|>")
        for pair in tqdm(qa_pairs[:int(1*len(qa_pairs))]):
            if pair.strip():  # Ensure the pair is not empty
                tokenized_pair = tokenizer.encode(pair, add_special_tokens=True, max_length=block_size, truncation=True)
                
            else:
                continue
                
            
            stand_for_index = self.find_sublist(tokenized_pair, stand_for_tokens)
            #print(stand_for_index,tokenized_pair)
             # Create a masked version of the tokenized pair
            masked_tokenized_pair = [-100] * len(tokenized_pair)
            if stand_for_index != -1:
                masked_tokenized_pair[stand_for_index + len(stand_for_tokens):] = tokenized_pair[stand_for_index + len(stand_for_tokens):]
                self.text.append(pair)
                self.examples.append(tokenized_pair)
                self.masked_examples.append(masked_tokenized_pair)
            else:
                print("pass")
                pass
                #for i in tokenized_pair:
                   # print(i,rev[i])
                    
        
    
    def find_sublist(self, main_list, sublist):
        """Find the start index of a sublist in a list."""
        length = len(sublist)
        for i in range(len(main_list) - length + 1):
            if main_list[i:i+length] == sublist:
                return i
        return -1
    
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return { "input_ids" : self.examples[item] }
        
train_file_path="train_524.txt"
model_name = 'models/Llama-3-8B-instruct'
output_dir = 'llama3-524-peft'
 
overwrite_output_dir = False
per_device_train_batch_size = 1
num_train_epochs = 1

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset = QADataset(train_file_path, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


QADataset
[128006, 78191, 128007]


100%|██████████| 1555/1555 [00:01<00:00, 1144.37it/s]


In [3]:
len(train_dataset)

1554

In [4]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig
from trl import SFTTrainer
from transformers import TrainingArguments
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
dataset = train_dataset


# Load the model + tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
response_template="<|start_header_id|>assistant<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


tokenizer.pad_token = tokenizer.eos_token
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    use_cache = False
)

# PEFT config
lora_alpha = 16
lora_dropout = 0.1
lora_r = 8
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["k_proj", "q_proj", "v_proj", "up_proj", "down_proj", "gate_proj"]
)


# Args 
max_seq_length = 2048*2
#output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 4
optim = "adamw_hf"
save_steps = 10
logging_steps = 1
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 580 # Approx the size of guanaco at bs 8, ga 2, 2 GPUs. 
warmup_ratio = 0.1
lr_scheduler_type = "cosine"
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    gradient_checkpointing=True,
    report_to="none",
)

# Trainer 
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    data_collator=collator,
)

# Not sure if needed but noticed this in https://colab.research.google.com/drive/1t3exfAVLQo4oKIopQT1SKxK4UcYg7rC1#scrollTo=7OyIvEx7b1GT
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# Train :)
trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/data/chenjiazun/.conda/envs/chenjiazun/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/data/chenjiazun/.conda/envs/chenjiazun/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos])

Step,Training Loss
1,3.559400
2,3.581800
3,2.693700
4,2.806300
5,3.282800
6,2.764800
7,2.962300
8,2.858400
9,2.459300
10,2.516400


/data/chenjiazun/.conda/envs/chenjiazun/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/data/chenjiazun/.conda/envs/chenjiazun/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/data/chenjiazun/.conda/envs/chenjiazun/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The def

TrainOutput(global_step=580, training_loss=0.28896882853048206, metrics={'train_runtime': 7089.99, 'train_samples_per_second': 0.654, 'train_steps_per_second': 0.082, 'total_flos': 6.786641286217728e+16, 'train_loss': 0.28896882853048206, 'epoch': 2.99})